In [1]:
import json
import re

In [2]:
# This file contains 8600ish users Data
# In the form: {'MBTI Type', 'Social Media Posts'}
dataFile = open('formatted_data.json', 'r')

In [3]:
data = []

In [4]:
# loading the json data into a list
for line in dataFile:
    data.append(json.loads(line))

In [5]:
types = []
posts = []

In [6]:
# Separating data into 2 separate lists for preprocessing
# For the most part we process the 'post' data
# we skip the first element as that is only the label
for i in range(1, len(data)):
    types.append(data[i]['Type'])
    #posts.append(data[i]['Post'])

In [7]:
import pickle

tokenized_posts = []
with open ('tokenized_formatted_data.txt', 'rb') as fp:
    tokenized_posts = pickle.load(fp)

In [8]:
all_words_list = []

for user in tokenized_posts:
    for word in user:
        all_words_list.append(word)

In [9]:
from collections import Counter

In [10]:
freq_list = Counter(all_words_list)
dictionary = freq_list.most_common(10000)

In [11]:
dictionary = list(zip(*dictionary))[0]

In [12]:
nums = range(0, 10000)
word_int = dict(zip(dictionary, nums))

In [13]:
x_vals = []

for user in tokenized_posts:
    x_vals.append([word_int[x] for x in user if x in word_int.keys()])

In [14]:
intro_extro = [word[0:1] for word in types[0:len(types)]]
bin_intro_extro = []
for letter in intro_extro:
    if (letter == 'I'):
        bin_intro_extro.append(0)
    else:
        bin_intro_extro.append(1)


In [15]:
import numpy as np
import random
random.seed(500)

x = np.array(x_vals)
random.shuffle(x)
test_data = x[:1500]
train_data = x[1500:]

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [16]:
from keras.utils.np_utils import to_categorical

y = np.asarray(bin_intro_extro).astype('float32')
random.shuffle(y)
y_test = y[:1500]
y_train = y[1500:]

Using TensorFlow backend.


In [17]:
x_val = x_train[:1500]
x_partial_train = x_train[1500:]

y_val = y_train[:1500]
y_partial_train = y_train[1500:]

In [ ]:
from keras import models 
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Embedding(10000, 32))
    model.add(layers.LSTM(32))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

model = build_model()
history = model.fit(x_partial_train, y_partial_train, epochs = 12, batch_size = 128)
results = model.evaluate(x_test, y_test)
#results